# TODO 

- Special keys treatment 
- user identification 
- improve logger (possibility to name the csv and to stop the app without restarting the kernel)
- 5 samples for each profile 
- Similarity measures (std, classification, nearest neighbor)

In [5]:
import pandas as pd 
import numpy as np
from datetime import datetime
from os import listdir
from os.path import isfile, join
import os
import math

# Data import from keylogger

In [6]:
#file_path = 'keylog.txt'
path_merged = "input_data_merged/"
path_preprocessed_merged = "preprocessed_input_data_merged/"
columns = ['Timestamp', 'Key', 'Action']
columns_bis = ['Timestamp', 'Key', 'Action', 'Shift']
#dataframe = pd.read_csv(file_path, sep=";", names=columns, encoding = "ISO-8859-1")
#display(dataframe)

# Preprocessing

In the event that a key is pressed for a 'long time' (at least 2 consecutive row with the same 'Key' value and 'Action' = press), we only keep the first pressed
<br>We also create some columns for specific keys like Shift, Alt, Win, Ctrl

In [7]:
path = 'input_data/'
user_list = files = [os.path.splitext(filename)[0] for filename in listdir(path)]
for i in range(0,len(user_list),5):
    all_filenames = []
    user_name = user_list[i].split('_')[0]
    for j in range(1, 6):    
        all_filenames = np.append(all_filenames, [path+user_name+'_'+str(j)+'.csv'])
    
    df_merged = (pd.read_csv(f, sep=';', names=columns, encoding = "ISO-8859-1") for f in all_filenames)
    df_merged = pd.concat(df_merged, ignore_index=False)
    df_merged.to_csv('input_data_merged/'+user_name+'.csv', sep='|', encoding = "ISO-8859-1")
    display(df_merged)

Timestamp          Key   Action
0    2021-11-03 09:21:29,850    Key.enter  release
1    2021-11-03 09:21:41,345    Key.shift    press
2    2021-11-03 09:21:41,473          'P'    press
3    2021-11-03 09:21:41,540          'P'  release
4    2021-11-03 09:21:41,601    Key.shift  release
..                       ...          ...      ...
958  2021-11-03 09:47:21,244          'e'  release
959  2021-11-03 09:47:21,901  Key.shift_r    press
960  2021-11-03 09:47:22,076          '.'    press
961  2021-11-03 09:47:22,142          '.'  release
962  2021-11-03 09:47:22,164  Key.shift_r  release

[5089 rows x 3 columns]

Timestamp        Key   Action
0    2021-11-03 08:44:53,007  Key.enter  release
1    2021-11-03 08:44:57,927  Key.shift    press
2    2021-11-03 08:44:58,035        'P'    press
3    2021-11-03 08:44:58,126        'P'  release
4    2021-11-03 08:44:58,180  Key.shift  release
..                       ...        ...      ...
981  2021-11-03 08:55:42,870        'e'  release
982  2021-11-03 08:55:42,986  Key.shift    press
983  2021-11-03 08:55:43,015        '.'    press
984  2021-11-03 08:55:43,124        '.'  release
985  2021-11-03 08:55:43,221  Key.shift  release

[4735 rows x 3 columns]

Timestamp         Key   Action
0     2021-11-10 09:57:34,370   Key.enter  release
1     2021-11-10 09:57:38,291   Key.shift    press
2     2021-11-10 09:57:38,518         'P'    press
3     2021-11-10 09:57:38,612         'P'  release
4     2021-11-10 09:57:38,642   Key.shift  release
...                       ...         ...      ...
1020  2021-11-10 10:10:22,361   Key.shift  release
1021  2021-11-10 10:10:22,606  Key.ctrl_l    press
1022  2021-11-10 10:10:22,933      '\x13'    press
1023  2021-11-10 10:10:23,056      '\x13'  release
1024  2021-11-10 10:10:23,205  Key.ctrl_l  release

[5133 rows x 3 columns]

Timestamp            Key   Action
0    2021-11-03 10:01:41,752      Key.enter  release
1    2021-11-03 10:01:58,027      Key.shift    press
2    2021-11-03 10:01:58,168            'P'    press
3    2021-11-03 10:01:58,302            'P'  release
4    2021-11-03 10:01:58,314      Key.shift  release
..                       ...            ...      ...
974  2021-11-03 10:13:26,554  Key.backspace  release
975  2021-11-03 10:13:26,632  Key.backspace    press
976  2021-11-03 10:13:26,708  Key.backspace  release
977  2021-11-03 10:13:27,223            '.'    press
978  2021-11-03 10:13:27,295            '.'  release

[4916 rows x 3 columns]

In [4]:
path_merged = "input_data_merged/"

def is_shifted_key(df, key_index):
    press = False
    release = False
    for i in range(key_index, df.index.start-1, -1):
        if df['Key'][i] == 'Key.shift' and df['Action'][i] == 'press':
            press = True
            for j in range(key_index, i, -1):
                if df['Key'][j] == 'Key.shift' and df['Action'][j] == 'release':
                    release = True
                    break
            break
    if press == True and release == False:
        return True
    else:
        return False

def preprocess_holding(dataframe):
    for i in range(dataframe.index.start, dataframe.index.stop):
        if i < dataframe.index.stop:
            current_key = dataframe['Key'][i]
            if dataframe['Action'][i] == 'press':
                press = True
                j = i + 1
                while press:
                    if j < dataframe.index.stop and dataframe['Action'][j] == 'press' and current_key == dataframe['Key'][j]:
                        dataframe = dataframe.drop([j])
                        dataframe = dataframe.reset_index(drop=True)
                    else:
                        break
    return dataframe
    
def preprocess_shift(dataframe):
    dataframe_bis = pd.DataFrame(columns=columns_bis)
    for i in range(dataframe.index.start, dataframe.index.stop):
        if is_shifted_key(dataframe, i):
            tmp = pd.DataFrame([[dataframe['Timestamp'][i], dataframe['Key'][i], dataframe['Action'][i], 1]], columns = columns_bis)
            dataframe_bis = dataframe_bis.append(tmp)
        else:
            tmp = pd.DataFrame([[dataframe['Timestamp'][i], dataframe['Key'][i], dataframe['Action'][i], 0]], columns = columns_bis)
            dataframe_bis = dataframe_bis.append(tmp)
    dataframe_bis = dataframe_bis.reset_index(drop=True)
    dataframe = dataframe_bis
    return dataframe

user_list = os.listdir(path_merged)
print(user_list)
for user in range(len(user_list)):
    dataframe = pd.read_csv(path_merged+user_list[user], sep="|", names=columns, encoding = "ISO-8859-1")
    dataframe = dataframe.reset_index(drop=True)
    dataframe = dataframe.drop([0])
    dataframe = dataframe.reset_index(drop=True)
    dataframe = preprocess_holding(dataframe)
    dataframe = preprocess_shift(dataframe)
    dataframe['Timestamp'] = pd.to_datetime(dataframe['Timestamp'], format='%Y-%m-%d %H:%M:%S,%f')  
    dataframe.to_csv(path_preprocessed_merged+user_list[user], sep='|', encoding = "ISO-8859-1")     
    display(dataframe)




['andrieu.csv', 'benjamin.csv', 'hugo.csv', 'johannes.csv']


Timestamp          Key   Action Shift
0    2021-11-03 09:21:29.850    Key.enter  release     0
1    2021-11-03 09:21:41.345    Key.shift    press     1
2    2021-11-03 09:21:41.473          'P'    press     1
3    2021-11-03 09:21:41.540          'P'  release     1
4    2021-11-03 09:21:41.601    Key.shift  release     0
...                      ...          ...      ...   ...
5066 2021-11-03 09:47:21.244          'e'  release     0
5067 2021-11-03 09:47:21.901  Key.shift_r    press     0
5068 2021-11-03 09:47:22.076          '.'    press     0
5069 2021-11-03 09:47:22.142          '.'  release     0
5070 2021-11-03 09:47:22.164  Key.shift_r  release     0

[5071 rows x 4 columns]

Timestamp        Key   Action Shift
0    2021-11-03 08:44:53.007  Key.enter  release     0
1    2021-11-03 08:44:57.927  Key.shift    press     1
2    2021-11-03 08:44:58.035        'P'    press     1
3    2021-11-03 08:44:58.126        'P'  release     1
4    2021-11-03 08:44:58.180  Key.shift  release     0
...                      ...        ...      ...   ...
4707 2021-11-03 08:55:42.870        'e'  release     0
4708 2021-11-03 08:55:42.986  Key.shift    press     1
4709 2021-11-03 08:55:43.015        '.'    press     1
4710 2021-11-03 08:55:43.124        '.'  release     1
4711 2021-11-03 08:55:43.221  Key.shift  release     0

[4712 rows x 4 columns]

Timestamp         Key   Action Shift
0    2021-11-10 09:57:34.370   Key.enter  release     0
1    2021-11-10 09:57:38.291   Key.shift    press     1
2    2021-11-10 09:57:38.518         'P'    press     1
3    2021-11-10 09:57:38.612         'P'  release     1
4    2021-11-10 09:57:38.642   Key.shift  release     0
...                      ...         ...      ...   ...
5112 2021-11-10 10:10:22.361   Key.shift  release     0
5113 2021-11-10 10:10:22.606  Key.ctrl_l    press     0
5114 2021-11-10 10:10:22.933      '\x13'    press     0
5115 2021-11-10 10:10:23.056      '\x13'  release     0
5116 2021-11-10 10:10:23.205  Key.ctrl_l  release     0

[5117 rows x 4 columns]

Timestamp            Key   Action Shift
0    2021-11-03 10:01:41.752      Key.enter  release     0
1    2021-11-03 10:01:58.027      Key.shift    press     1
2    2021-11-03 10:01:58.168            'P'    press     1
3    2021-11-03 10:01:58.302            'P'  release     1
4    2021-11-03 10:01:58.314      Key.shift  release     0
...                      ...            ...      ...   ...
4878 2021-11-03 10:13:26.554  Key.backspace  release     0
4879 2021-11-03 10:13:26.632  Key.backspace    press     0
4880 2021-11-03 10:13:26.708  Key.backspace  release     0
4881 2021-11-03 10:13:27.223            '.'    press     0
4882 2021-11-03 10:13:27.295            '.'  release     0

[4883 rows x 4 columns]

In [115]:
dataframe = pd.read_csv("test.csv", sep = "|", names=columns, encoding = "ISO-8859-1")
dataframe = preprocess_holding(dataframe)
dataframe = preprocess_shift(dataframe)
dataframe['Timestamp'] = pd.to_datetime(dataframe['Timestamp'], format='%Y-%m-%d %H:%M:%S,%f')  
dataframe.to_csv("test_preprocessed.csv", sep='|', encoding = "ISO-8859-1")     
display(dataframe)

Timestamp        Key   Action Shift
0    2021-11-10 10:15:45.341  Key.enter  release     0
1    2021-11-10 10:15:47.896  Key.shift    press     1
2    2021-11-10 10:15:48.110        'P'    press     1
3    2021-11-10 10:15:48.198        'P'  release     1
4    2021-11-10 10:15:48.227  Key.shift  release     0
...                      ...        ...      ...   ...
1046 2021-11-10 10:17:31.222        'l'  release     0
1047 2021-11-10 10:17:31.290        'l'    press     0
1048 2021-11-10 10:17:31.380        'l'  release     0
1049 2021-11-10 10:17:31.382        'e'    press     0
1050 2021-11-10 10:17:31.473        'e'  release     0

[1051 rows x 4 columns]

## Reference Base Building

**columns_nd** = *nd for normalize data* <br>
**Hold Duration (HD)** = *Duration between press and release of the same key* <br>
**Hold time in digraph (HTD)** = *Dwell time with the context of the preceding keystroke* <br/>
**Cumulative Dwell Time (CDT)** = *Holding time for two consecutive keys without flying time* <br>
**Digraph latency (DL)** = *Flight time between two keystrokes*





In [116]:

columns_nd = ['Key1', 'Key2', 'HD', 'HTD', 'CDT', 'DL', 'Shift']# 'Shift', 'Alt', 'Ctrl', 'Win']
df = pd.DataFrame(columns = columns_nd)
df_bis = pd.DataFrame(columns = columns_nd)
columns_ref = ["Key", "Shift", "HD_mean", "HD_std", "DL_mean", "DL_std"]
path_reference = "reference_1/"
date_time_str = '2021-11-03 10:13:00.358'

date_time_obj = datetime. strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')

def search_for_press(df):
    for i in range(df.index.start, df.index.stop):
        if df['Action'][i] == 'press':
            return i

def search_for_release(df, press_index):
    for i in range (press_index, df.index.stop):
        if (df['Key'][i] == df['Key'][press_index] or df['Key'][i].lower() == df['Key'][press_index].lower()) and df['Action'][i] == 'release':
            return i
        
def search_for_following_key(df, key_index):
    for i in range(key_index, df.index.stop):
        if df['Key'][i] != df['Key'][key_index] and df['Action'][i] == 'press':
            return i
def metrics_computation(dataframe):
    df = pd.DataFrame(columns = columns_nd)                  
    while dataframe.empty == False:
        end = False
        if dataframe['Action'][0] == 'release':
            dataframe = dataframe.drop([0])
            dataframe = dataframe.reset_index(drop=True)

        press_index = search_for_press(dataframe)
        release_index = search_for_release(dataframe, press_index)
        
        if dataframe['Timestamp'][release_index] == date_time_obj:
            print(dataframe['Timestamp'][press_index])
        
        if dataframe.index.stop-dataframe.index.start <= 2:
            end = True
        if end == False:
            following_key_press_index = search_for_following_key(dataframe, press_index)
            following_key_release_index = search_for_release(dataframe, following_key_press_index)
            Key1 = dataframe['Key'][press_index]
            Key2 = dataframe['Key'][following_key_press_index]
            print(dataframe['Timestamp'][following_key_press_index])

            print(dataframe['Timestamp'][following_key_release_index])
            HD = dataframe['Timestamp'][release_index] - dataframe['Timestamp'][press_index]
            HD = HD.total_seconds()
            HTD = dataframe['Timestamp'][following_key_release_index] - dataframe['Timestamp'][following_key_press_index]
            HTD = HTD.total_seconds()
            CDT = HD + HTD
            DL = dataframe['Timestamp'][following_key_press_index] - dataframe['Timestamp'][release_index]
            DL = DL.total_seconds()
            shift = dataframe['Shift'][press_index]
            dataframe = dataframe.drop([press_index, release_index])
            dataframe = dataframe.reset_index(drop=True)
            tmp = pd.DataFrame([[Key1, Key2, HD, HTD, CDT, DL, shift]], columns = columns_nd)
            df = df.append(tmp)
        else:
            Key1 = dataframe['Key'][press_index]
            HD = dataframe['Timestamp'][release_index] - dataframe['Timestamp'][press_index]
            HD = HD.total_seconds()
            HTD = None
            CDT = None
            DL = None
            shift = dataframe['Shift'][press_index]
            dataframe = dataframe.drop([press_index, release_index])
            tmp = pd.DataFrame([[Key1, None, HD, HTD, CDT, DL, shift]], columns = columns_nd)
            df = df.append(tmp)
    df = df.reset_index(drop=True)
    return df

def compute_reference(dataframe):
    df_ref = pd.DataFrame(columns = columns_ref)
    while dataframe.empty == False:
        hold_durations = []
        flight_durations = []
        key_save = dataframe['Key1'][0]
        shift_save = dataframe["Shift"][0]
        for i in dataframe.index:
            if i < dataframe.index.stop:
                if dataframe['Key1'][i] == key_save and shift_save == dataframe['Shift'][i]:
                    hold_durations = np.append(hold_durations, dataframe['HD'][i])
                    if i != dataframe.index.stop - 1:
                        flight_durations = np.append(flight_durations, dataframe['DL'][i])
                    dataframe = dataframe.drop([i])
                    dataframe = dataframe.reset_index(drop=True)       
        mean_HD = np.mean(hold_durations)
        std_HD = np.std(hold_durations)
        mean_DL = np.mean(hold_durations)
        std_DL = np.std(flight_durations)
        result = pd.DataFrame([[key_save, shift_save, mean_HD, std_HD, mean_DL, std_DL]], columns = columns_ref)
        df_ref = df_ref.append(result)
    return df_ref


        

user_list = os.listdir(path_preprocessed_merged)       
print(user_list)
for user in range(len(user_list)):
    dataframe = pd.read_csv(path_preprocessed_merged+user_list[user], sep="|", names=columns_bis, encoding = "ISO-8859-1")
    dataframe = dataframe.reset_index(drop=True)
    dataframe = dataframe.drop([0])
    dataframe = dataframe.reset_index(drop=True)
    dataframe['Timestamp'] = pd.to_datetime(dataframe['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')   
    df_bis = metrics_computation(dataframe)
    display(df_bis)
    df_ref = compute_reference(df_bis)
    df_ref = df_ref.reset_index(drop=True)
    display(df_ref)
    df_ref.to_csv(path_reference+user_list[user], sep='|', encoding = "ISO-8859-1")     

    
       





['andrieu.csv', 'benjamin.csv', 'hugo.csv', 'johannes.csv']
2021-11-03 09:21:41.473000
2021-11-03 09:21:41.540000
2021-11-03 09:21:41.755000
2021-11-03 09:21:41.821000
2021-11-03 09:21:41.856000
2021-11-03 09:21:41.946000
2021-11-03 09:21:42.057000
2021-11-03 09:21:42.123000
2021-11-03 09:21:42.319000
2021-11-03 09:21:42.406000
2021-11-03 09:21:42.527000
2021-11-03 09:21:42.612000
2021-11-03 09:21:42.910000
2021-11-03 09:21:43.003000
2021-11-03 09:21:43.115000
2021-11-03 09:21:43.180000
2021-11-03 09:21:43.280000
2021-11-03 09:21:43.349000
2021-11-03 09:21:43.321000
2021-11-03 09:21:43.385000
2021-11-03 09:21:43.532000
2021-11-03 09:21:43.594000
2021-11-03 09:21:43.636000
2021-11-03 09:21:43.700000
2021-11-03 09:21:43.737000
2021-11-03 09:21:43.800000
2021-11-03 09:21:43.861000
2021-11-03 09:21:43.928000
2021-11-03 09:21:43.958000
2021-11-03 09:21:44.042000
2021-11-03 09:21:44.109000
2021-11-03 09:21:44.174000
2021-11-03 09:21:45.202000
2021-11-03 09:21:45.264000
2021-11-03 09:21:45.29

KeyError: None

In [43]:
dataframe = pd.read_csv("test_preprocessed.csv", sep="|", names=columns_bis, encoding = "ISO-8859-1")
dataframe = dataframe.reset_index(drop=True)
dataframe = dataframe.drop([0])
dataframe = dataframe.reset_index(drop=True)
display(dataframe)
dataframe['Timestamp'] = pd.to_datetime(dataframe['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')   
df_bis = metrics_computation(dataframe)
display(df_bis)
df_ref = compute_reference(df_bis)
df_ref = df_ref.reset_index(drop=True)
display(df_ref)
df_ref.to_csv("test_reference.csv", sep='|', encoding = "ISO-8859-1") 

Timestamp        Key   Action Shift
0     2021-11-10 10:15:45.341  Key.enter  release     0
1     2021-11-10 10:15:47.896  Key.shift    press     1
2     2021-11-10 10:15:48.110        'P'    press     1
3     2021-11-10 10:15:48.198        'P'  release     1
4     2021-11-10 10:15:48.227  Key.shift  release     0
...                       ...        ...      ...   ...
1046  2021-11-10 10:17:31.222        'l'  release     0
1047  2021-11-10 10:17:31.290        'l'    press     0
1048  2021-11-10 10:17:31.380        'l'  release     0
1049  2021-11-10 10:17:31.382        'e'    press     0
1050  2021-11-10 10:17:31.473        'e'  release     0

[1051 rows x 4 columns]

Key1  Key2     HD    HTD    CDT     DL Shift
0    Key.shift   'P'  0.331  0.088  0.419 -0.117     1
1          'P'   'o'  0.088  0.089  0.177  0.103     1
2          'o'   'r'  0.089  0.064  0.153  0.036     0
3          'r'   't'  0.064  0.092  0.156   0.06     0
4          't'   'e'  0.092  0.096  0.188   0.04     0
..         ...   ...    ...    ...    ...    ...   ...
520        'v'   'i'  0.093  0.092  0.185  0.002     0
521        'i'   'l'  0.092  0.063  0.155  0.094     0
522        'l'   'e'  0.063  0.091  0.154   0.16     0
523        'l'   'e'  0.090  0.091  0.181  0.002     0
524        'e'  None  0.091   None   None   None     0

[525 rows x 7 columns]

C:\Users\hugod\anaconda3\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\hugod\anaconda3\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
C:\Users\hugod\anaconda3\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Key Shift   HD_mean    HD_std   DL_mean    DL_std
0   Key.shift     1  0.339885  0.173445  0.339885  0.130820
1         'P'     1  0.088000  0.000000  0.088000  0.000000
2         'o'     0  0.092643  0.020838  0.092643  0.136093
3         'r'     0  0.092353  0.025079  0.092353  0.167735
4         't'     0  0.083875  0.013043  0.083875  0.033630
..        ...   ...       ...       ...       ...       ...
61        'S'     1  0.092000  0.000000  0.092000  0.000000
62        '-'     0  0.060000  0.000000  0.060000  0.000000
63        '-'     0  0.060000  0.000000  0.060000  0.000000
64        'n'     0  0.124000  0.000000  0.124000  0.000000
65        'z'     0  0.153000  0.000000  0.153000       NaN

[66 rows x 6 columns]

# Scaled Manhattan Verifier

In [95]:
path = 'reference_1/'
user_list = files = [os.path.splitext(filename)[0] for filename in listdir(path)]
print(user_list)
def scaled_manhattan_verifier(vector):
    distance_to_users = np.array([[0,0]])
    distance_to_users = np.delete(distance_to_users, 0, 0)
    index = 0

    for user in range(len(user_list)):
        user_ref = pd.read_csv(path+user_list[user]+".csv", sep="|", names=columns_ref, encoding = "ISO-8859-1")
        user_ref = user_ref.reset_index(drop=True)
        user_ref = user_ref.drop([0])
        user_ref = user_ref.reset_index(drop=True)
        distance = 0
        for i in range(vector.shape[0]):
            for j in range(user_ref.shape[0]):
                if user_ref['Key'][j] == vector['Key'][i]:
                   if float(user_ref['HD_std'][j]) != 0 and math.isnan(float(user_ref['HD_std'][j])) == False:
                    distance += abs(float(vector['HD_mean'][i]) - float(user_ref['HD_mean'][j]))/float(user_ref['HD_std'][j])
                   if float(user_ref['DL_std'][j]) != 0 and math.isnan(float(user_ref['DL_std'][j])) == False:
                    distance += abs(float(vector['DL_mean'][i]) - float(user_ref['DL_mean'][j]))/float(user_ref['DL_std'][j])
        distance_to_users = np.append(distance_to_users, [[index, distance]], axis = 0)
        index += 1
        distance = 0
    minim = float('inf')
    res = 0
    display(distance_to_users)
    for i in range(len(distance_to_users)):
        if distance_to_users[i][1] < minim:
            minim = distance_to_users[i][1]
            res = i
    print(res)
    print('Vous êtes '+user_list[res])
                    
vector = pd.read_csv("test_reference.csv", sep="|", names=columns_ref, encoding = "ISO-8859-1")
vector = vector.reset_index(drop=True)
vector = vector.drop([0])
vector = vector.reset_index(drop=True)
display(vector)
scaled_manhattan_verifier(vector)     
            
            

['benjamin', 'hugo', 'johannes']


Key Shift              HD_mean                HD_std  \
0   Key.shift     1  0.33988461538461545   0.17344547512182812   
1         'P'     1                0.088                   0.0   
2         'o'     0  0.09264285714285715   0.02083817630784265   
3         'r'     0  0.09235294117647058  0.025078561682348594   
4         't'     0             0.083875  0.013042598475763945   
..        ...   ...                  ...                   ...   
61        'S'     1                0.092                   0.0   
62        '-'     0                 0.06                   0.0   
63        '-'     0                 0.06                   0.0   
64        'n'     0                0.124                   0.0   
65        'z'     0                0.153                   0.0   

                DL_mean               DL_std  
0   0.33988461538461545  0.13081962829762586  
1                 0.088                  0.0  
2   0.09264285714285715  0.13609278047699364  
3   0.09235294117647058  0.16773456952324645  
4              0.083875  0.03363034344160047  
..                  ...                  ...  
61                0.092                  0.0  
62                 0.06                  0.0  
63                 0.06                  0.0  
64                0.124                  0.0  
65                0.153                  NaN  

[66 rows x 6 columns]

array([[  0.        , 160.4782387 ],
       [  1.        , 125.28812775],
       [  2.        , 160.4782387 ]])

1
Vous êtes hugo
